In [1]:
import pandas as pd

In [2]:
import gzip
import json

In [3]:
def parse(path): 
    g = gzip.open(path, 'rb') 
    for l in g: 
        yield eval(l) 

In [4]:
def getDF(path): 
    i = 0 
    df = {}
    for d in parse(path): 
        df[i] = d 
        i += 1 
    return pd.DataFrame.from_dict(df, orient='index') 

In [5]:
df = getDF('reviews_Cell_Phones_and_Accessories_5.json.gz')

In [6]:
newDf = df[['reviewText','overall']]

In [7]:
newDf.head()

,reviewText,overall
0,They look good and stick good! I just don't li...,4.0
1,These stickers work like the review says they ...,5.0
2,These are awesome and make my phone look so st...,5.0
3,Item arrived in great time and was in perfect ...,4.0
4,"awesome! stays on, and looks great. can be use...",5.0


In [8]:
newDf['overall'] = newDf['overall'].apply(lambda score : 'pos' if score >= 3.0 else 'neg')

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [9]:
import nltk

In [10]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

In [11]:
newDf['scores'] = newDf['reviewText'].apply(lambda review : sid.polarity_scores(review))

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [12]:
newDf.head()

,reviewText,overall,scores
0,They look good and stick good! I just don't li...,pos,"{'neg': 0.182, 'neu': 0.678, 'pos': 0.14, 'com..."
1,These stickers work like the review says they ...,pos,"{'neg': 0.0, 'neu': 0.624, 'pos': 0.376, 'comp..."
2,These are awesome and make my phone look so st...,pos,"{'neg': 0.0, 'neu': 0.762, 'pos': 0.238, 'comp..."
3,Item arrived in great time and was in perfect ...,pos,"{'neg': 0.0, 'neu': 0.686, 'pos': 0.314, 'comp..."
4,"awesome! stays on, and looks great. can be use...",pos,"{'neg': 0.0, 'neu': 0.643, 'pos': 0.357, 'comp..."


In [13]:
newDf['compound'] = newDf['scores'].apply(lambda d:d['compound'])

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [14]:
newDf.head()['scores'][0]

{'neg': 0.182, 'neu': 0.678, 'pos': 0.14, 'compound': -0.1808}

In [15]:
newDf['new_rating'] = newDf['compound'].apply(lambda score: 'pos' if score >=0 else 'neg')

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [16]:
newDf.head()

,reviewText,overall,scores,compound,new_rating
0,They look good and stick good! I just don't li...,pos,"{'neg': 0.182, 'neu': 0.678, 'pos': 0.14, 'com...",-0.1808,neg
1,These stickers work like the review says they ...,pos,"{'neg': 0.0, 'neu': 0.624, 'pos': 0.376, 'comp...",0.9403,pos
2,These are awesome and make my phone look so st...,pos,"{'neg': 0.0, 'neu': 0.762, 'pos': 0.238, 'comp...",0.8852,pos
3,Item arrived in great time and was in perfect ...,pos,"{'neg': 0.0, 'neu': 0.686, 'pos': 0.314, 'comp...",0.9625,pos
4,"awesome! stays on, and looks great. can be use...",pos,"{'neg': 0.0, 'neu': 0.643, 'pos': 0.357, 'comp...",0.9020,pos


In [17]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report

In [18]:
print(accuracy_score(newDf['overall'],newDf['new_rating']))

0.8532907492838371


In [19]:
print(classification_report(newDf['overall'],newDf['new_rating']))

              precision    recall  f1-score   support

         neg       0.42      0.48      0.45     24343
         pos       0.92      0.91      0.92    170096

   micro avg       0.85      0.85      0.85    194439
   macro avg       0.67      0.69      0.68    194439
weighted avg       0.86      0.85      0.86    194439



In [20]:
print(confusion_matrix(newDf['overall'],newDf['new_rating']))

[[ 11643  12700]
 [ 15826 154270]]
